# **Literal Object**

In [0]:
class Literal(object):
    """Represents a literal in a clause.
    
    Parameters
    ----------
    name : str
        A str to represent to name of the object.
    sign : bool
        Indicates whether the literal is negated
        
    """
    def __init__(self,name='',sign=True):
        self.name = name
        self.sign = sign  # True if literal is positive, else False

    def __repr__(self):
        return self.__str__()

    def __str__(self):
        # print sign and name of literal
        if self.sign == True:
            sign = ''
        else:
            sign = '-'
        return sign + self.name
      
    def __eq__(self, other):
        if isinstance(other, basestring):  # equating to str object
            return self.name == other
        if isinstance(other, set):  # equating to set object
            return self.name == other
        return self.name == other.name  # independent of sign
      
    def __hash__(self):
        # hash is independent of sign
        return hash(self.name)
      
    def __neg__(self):
        # create new instance with negated literal 
        return Literal(self.name, not self.sign)

# **Simple DPLL Implementation**

In [0]:
from copy import deepcopy

def DPLLSatisfiable_Simple(KB):
    """ Tests if a KB is satisfiable
    
    Input: a KB is a CNF in the form of list of sets
           where each set represents a disjunctive clause
           and clauses are conjunctive to one another
           e.g. [{A,B},{A,-C},{-A,B,D}]
           
    Outputs: 
           sat = Satisfiability of KB (True or False)
           truth_assignment = Truth assignment of the satisfiable model
                              (Dictionary with symbols as keys, and their truth 
                               assignments - 'True', 'False' or 'Free')
    """
    global truth_assignment
    truth_assignment = {}  # a global variable storing the working model
    
    true_literal = Literal('True') # assigned to sets with true literals
        
    full_symbols = {}   # a dictionary of symbols and their frequencies
    
    # Updating full_symbols with the frequency of each literal in the given KB
    for clause in KB:
        for literal in clause:
            if literal in full_symbols:
                full_symbols[literal] += 1
            else:
                full_symbols[literal] = 1
    
    # Degree heuristic:
    # sorted keys in descending order based on frequency
    # first element, which has highest frequency, to be popped first
    full_symbols = sorted([(freq, key) for key, freq in full_symbols.items()],
                         key=lambda x: x[0], reverse = True)
    
    # retain keys as a list and drop frequency counts
    full_symbols = [s[1] for s in full_symbols]
    
    def DPLL_Simple(clauses,symbols,model):  
        '''Recursively searches for a model which satisfies the CNF-SAT problem
        
        Inputs:
              clauses = the KB in list of sets
              symbols = list of symbols, where a symbol is popped each recursion
              model = dictionary of the current model 
              
        Outputs:
              sat = Satisfiability of clauses (True or False)
              truth_assignment = Truth assignment of the satisfiable model
              
        '''
        # Update the clauses based on the current model
        for clause in clauses:
            deleted_symbols = set()  # store symbols we can remove from a clause
            for literal in clause:
                if literal.name in model:
                    # If sign of a literal matches what is in the model,
                    # the literal is True and therefore the clause is True
                    if ((model[literal.name] and literal.sign) 
                        or (not model[literal] and not literal.sign)):
                        deleted_symbols.update(clause)  # union set
                        clause.add(true_literal)
                        break
                    # If clause has other elements and literal is False, 
                    # remove the literal from the clause since x or False = x
                    elif len(clause) > 1:
                        deleted_symbols.add(literal)
                    # Backtrack if the clause failed and assign False to symbols
                    else:
                        return False, None

            # Remove deleted_symbols from the clause
            clause.difference_update(deleted_symbols)
                       
        # If every clause evaluates to True, return True and truth_assignment
        if all([true_literal in clause for clause in clauses]):
            global truth_assignment
            truth_assignment = model
            return True, truth_assignment

        # Recurse until all symbols are popped for True and False assignments
        if symbols is not None:          
            p = symbols.pop(0)
            # print "Popping symbol: {}".format(p)
            
            # Making deep copies of clauses, symbols and model
            true_clause = deepcopy(clauses)
            false_clause = deepcopy(clauses)
            
            true_model_symbols = deepcopy(full_symbols)
            false_model_symbols = deepcopy(full_symbols)
            
            true_model = deepcopy(model)
            false_model = deepcopy(model)
                      
            true_model[p.name] = True  
            false_model[p.name] = False      
            
            # Recursing into True or False sub-trees at each symbol            
            sat, truth_assignment = DPLL_Simple(true_clause,true_model_symbols,true_model)
            if sat:
                return sat, truth_assignment
                              
            sat, truth_assignment = DPLL_Simple(false_clause,false_model_symbols,false_model)
            if sat:
                return sat, truth_assignment
              
        # All symbols exhausted and solution not found
        return False, None
    
    sat, solution = DPLL_Simple(KB,full_symbols,{})
    
    if sat == True:
        # Find literals that are not in truth_assignment
        for clauses in KB:
          for literal in clauses:
              # Update its key with 'Free' if they can be True or False
              if literal.name not in truth_assignment:
                  truth_assignment[literal.name] = "Free"
              # For literals that are True or False, update boolean to strings
              # as specified in example output of the assignment
              else:
                  truth_assignment[literal.name] = str(truth_assignment[literal.name])

    mem_usage = memory_usage(max_usage = True)

    return sat, truth_assignment, mem_usage

# **Extended DPLL Implementation**

In [0]:
def DPLLSatisfiable(KB):
    """Tests if a KB is satisfiable
    
    Input: a KB is a CNF in the form of list of sets
           where each set represents a disjunctive clause
           and clauses are conjunctive to one another
           e.g. [{A,B},{A,-C},{-A,B,D}]
           
    Outputs: 
           sat = Satisfiability of KB (True or False)
           truth_assignment = Truth assignment of the satisfiable model
                              (Dictionary with symbols as keys, and their truth 
                               assignments - 'True', 'False' or 'Free')
    """
    global truth_assignment
    truth_assignment = {}  # a global variable storing the working model
    
    true_literal = Literal('True') # assigned to sets with true literals
    
    def find_unit_clause(clauses,model):
        '''Finds a unit clause in a list of clauses
        A unit clause is a clause with length 1
        '''
        for clause in clauses:
            # find a unit clause that is not already True
            if len(clause) == 1 and true_literal not in clause:
                for literal in clause:
                    return min(clause), literal.sign

        # else return None
        return None, None
    
    def find_pure_symbol(symbols,clauses,model):
        '''Finds a pure symbol in a list of clauses
        A pure symbol is when a literal appears only in one sign in all clauses
        Returns True if found, followed by the symbol itself; else None
        '''
        def check_if_pure(symbol):
            '''Checks if an input symbol is pure 
            Sub-function breaks early if symbol not found
            '''
            pure_symbol = None  # initial pure_symbol is None
            # Check for each literal in KB for the specified symbol
            for clause in clauses:
                if symbol in clause:
                    for literal in clause:
                        if literal == symbol:
                            # Pure_symbol not found if their signs do not match
                            # Iterate to next symbol since pure_symbol = None
                            if pure_symbol is not None and literal.sign != pure_symbol:
                                return False, None
                            pure_symbol = literal.sign
            return True, pure_symbol

        # Call function check_if_pure for each symbol in KB
        for symbol in symbols:
            if_pure, found_pure_symbol = check_if_pure(symbol)
            # Return symbol and its corresponding sign
            if if_pure and found_pure_symbol is not None:
                return symbol, found_pure_symbol
        
        # Else, no pure symbol found
        return None, None

    def DPLL(clauses,symbols,model):  
        '''Recursively searches for a model which satisfies the CNF-SAT problem
        
        Inputs:
              clauses = the KB in list of sets
              symbols = list of symbols, where a symbol is popped each recursion
              model = dictionary of the current model 
              
        Outputs:
              sat = Satisfiability of clauses (True or False)
              truth_assignment = Truth assignment of the satisfiable model
              
        '''
        # Update the clauses based on the current model
        for clause in clauses:
            deleted_symbols = set()  # store symbols we can remove from a clause
            for literal in clause:
                if literal.name in model:
                    # If sign of a literal matches what is in the model,
                    # the literal is True and therefore the clause is True
                    if ((model[literal.name] and literal.sign) 
                        or (not model[literal] and not literal.sign)):
                        deleted_symbols.update(clause)  # union set
                        clause.add(true_literal)
                        break
                    # If clause has other elements and literal is False, 
                    # remove the literal from the clause since x or False = x
                    elif len(clause) > 1:
                        deleted_symbols.add(literal)
                    # Backtrack if the clause failed and assign False to symbols
                    else:
                        return False, None

            # Remove deleted_symbols from the clause
            clause.difference_update(deleted_symbols)
                                      
        # If every clause evaluates to True, return True and truth_assignment
        if all([true_literal in clause for clause in clauses]):
            global truth_assignment
            truth_assignment = model
            return True, truth_assignment
          
        # Recurse until all symbols are popped for True and False assignments
        if symbols is not None:    
          
            # Run pure symbol heuristic
            pure_symbol, sign = find_pure_symbol(full_symbols,clauses,model)

            if pure_symbol is not None:
                # Prune pure_symbol since pure_symbol OR literals
                # we know the sign of pure_symbol, if True the clause is True
                # Otherwise, pure_symbol is removed from the clause
                # print pure_symbol, "is a pure symbol"
                symbols.remove(pure_symbol)
                # Add pure_symbol and its sign to the working model
                # print "Assigning {} pure symbol to {}".format(pure_symbol,sign)
                model.update({pure_symbol.name: sign})
                return DPLL(deepcopy(clauses), symbols, model)

            # Run unit clause heuristic
            unit, unit_sign = find_unit_clause(clauses, model)

            if unit is not None:
                # Prune using unit propagation
                # print "Unit propagation on unit:", unit
                symbols.remove(unit.name)
                model.update({unit.name: unit_sign})
                return DPLL(deepcopy(clauses), symbols, model)

            # Run degree heuristics            
            p = symbols.pop(0)
            # print "Popping symbol: {}".format(p)
            
            # Making deep copies of clauses, symbols and model
            true_clause = deepcopy(clauses)
            false_clause = deepcopy(clauses)
            
            true_model_symbols = deepcopy(full_symbols)
            false_model_symbols = deepcopy(full_symbols)
            
            true_model = deepcopy(model)
            false_model = deepcopy(model)
                      
            true_model[p.name] = True  
            false_model[p.name] = False      
            
            # Recursing into True or False sub-trees at each symbol            
            sat, truth_assignment = DPLL(true_clause,true_model_symbols,true_model)
            if sat:
                return sat, truth_assignment
                              
            sat, truth_assignment = DPLL(false_clause,false_model_symbols,false_model)
            if sat:
                return sat, truth_assignment
              
        # All symbols exhausted and solution not found
        return False, None

    full_symbols = {}   # a dictionary of symbols and their frequencies
    
    # Updating full_symbols with the frequency of each literal in the given KB
    for clause in KB:
        for literal in clause:
            if literal in full_symbols:
                full_symbols[literal] += 1
            else:
                full_symbols[literal] = 1
    
    # Degree heuristic:
    # sorted keys in descending order based on frequency
    # first element, which has highest frequency, to be popped first
    full_symbols = sorted([(freq, key) for key, freq in full_symbols.items()],
                         key=lambda x: x[0], reverse = True)
    
    # retain keys as a list and drop frequency counts
    full_symbols = [s[1] for s in full_symbols]
    
    sat, solution = DPLL(KB,full_symbols,{})
      
    if sat:
        # Find literals that are not in truth_assignment
        for clauses in KB:
          for literal in clauses:
              # Update its key with 'Free' if they can be True or False
              if literal.name not in truth_assignment:
                  truth_assignment[literal.name] = "Free"
              # For literals that are True or False, update boolean to strings
              # as specified in example output of the assignment
              else:
                  truth_assignment[literal.name] = str(truth_assignment[literal.name])
                  
    mem_usage = memory_usage(max_usage = True)

    return sat, truth_assignment, mem_usage

# **Testing DPLL on Textbook KB**

In [4]:
import time
from memory_profiler import memory_usage

A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, 
      {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]

start_time = time.time()
satisfiable, model, mem_usage = DPLLSatisfiable(KB)
print (satisfiable, model)
time_taken = time.time() - start_time
print "Time taken: %.2f s" % time_taken

print "Memory used: %.2f MB" % mem_usage

(True, {'A': 'False', 'C': 'Free', 'B': 'False', 'E': 'False', 'D': 'True', 'F': 'Free'})
Time taken: 0.10 s
Memory used: 131.92 MB
